In [27]:
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import numpy as np
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
# import selenium
from selenium import webdriver
import undetected_chromedriver as uc
import random
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import WebDriverException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException
import requests

C:\Users\USER\AppData\Local\Temp\ipykernel_13792\2069570731.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [29]:
# Incognito Chrome

'''chrome_options = Options()
chrome_options.add_experimental_option("detach", True)
chrome_options.headless = False # For debugging. SHould be set to True when running in an actual test case
chrome_options.add_experimental_option("excludeSwitches",["enable-logging"])
chrome_options.add_argument("--incognito")

driver = webdriver.Chrome(options=chrome_options)'''

In [3]:
print(selenium.__version__)

4.17.2


* Summary:

1) Bloomberg는 cookies acceptance 라는 Javascript rendering 때문에 Zenrows API를 사용하였습니다!

2) Yahoo Finance랑 Wall Street Journal은 Undetected Chromedriver로 해야 verification이 패스가 되고 크롤링이 되서 이걸로 하였습니다!

3) Financial Times나 New York Times은 제가 free article limit을 reach했는지는 몰라도 embedded paywall + sign-up 때문에 그냥 기사조차 안들어가집니다 (ㅠㅠ)

4) CNBC랑 Forbes는 Apify API를 사용하였습니다! 이 두 뉴스기사들은 왜 Apify API를 썼냐고 궁금하실수있으실겉같은데, 얘네도 실은 Zenrows API, Undetected Chromedriver 둘 다 가능합니다! 

--> 하지만 Zenrows API는 API credit이 많이나가서 Javascript rendering이 없는, 예를 들어서, cookies acceptance 팝업이 없는 경우에는 Zenrows API credit을 굳이 써야할까 싶어서 쓰지않았습니다..!

--> Undetected Chromedriver는 verification이 필요한 경우에는 유용하지만, verification이 필요없는 뉴스기사들 같은 경우에는 이걸 굳이 안써도 될것같은게 스피드가 조금 느립니다ㅠㅠ 

--> 그래서 대안으로 Apify API를 갖고와봤는데 얘는 속도도 빠르고 좋지만 Javascript rendering, verification이 필요한 뉴스 웹사이트들에는 먹히지않습니다ㅠㅠ

* Bloomberg (ZenRows is used to deal with Javascript renderings)

In [56]:
# pip install requests beautifulsoup4

import requests
from zenrows import ZenRowsClient
from bs4 import BeautifulSoup

client = ZenRowsClient("--API KEY")
url = "https://www.bloomberg.com/news/articles/2024-02-11/five-key-charts-to-watch-in-global-commodity-markets-this-week?srnd=industries-v2"
params = {"js_render":"true","block_resources":"media,xhr,eventsource,image"}

response = client.get(url, params=params)
soup = BeautifulSoup(response.text, "html.parser")

blb_news = pd.DataFrame(columns=['URL','h1','text'])

title = soup.find("h1").text
p_tags = soup.find_all('p', attrs={'class': ['Paragraph_text-SqIsdNjh0t0-', 'Paragraph_text-SqIsdNjh0t0- paywall']})

text_content = ""
for p in p_tags:
    text_content += p.text + " "
text_content = text_content.strip()

blb_news = blb_news._append(
    {"URL": url,
     "h1": title,
     "text": text_content},
    ignore_index=True
)


In [57]:
blb_news

,URL,h1,text
0,https://www.bloomberg.com/news/articles/2024-0...,Five Key Charts to Watch in Global Commodity M...,It’s bad timing for chocolate lovers with coco...


In [58]:
blb_news['text'][0]

'It’s bad timing for chocolate lovers with cocoa prices soaring to the highest ever. Meanwhile, record warm temperatures look set to continue, posing an ongoing threat to crops, energy supplies and transportation. The Organization of Petroleum Exporting Countries and the International Energy Agency publish their respective monthly oil market reports. And more earnings are on tap, including from mining behemoth Barrick Gold Corp., machinery maker Deere & Co. and oil companies Eni SpA and Occidental Petroleum Corp. Here are five notable charts to consider in global commodity markets as the week gets underway.'

* Yahoo Finance (Undetected Chromedriver is used. Apify API does not work with this news source for some reason..)

In [31]:
def get_yf_news(url):

    '''driver = webdriver.Chrome(options=chrome_options)
    driver.get(url)
    driver.implicitly_wait(2)'''

    options = webdriver.ChromeOptions()
    options.add_argument('proxy-server=172.67.134.0:80')
    options.add_argument("--disable-cache")
    browser = uc.Chrome(
        options = options
    )
    browser.get(url)
    browser.implicitly_wait(2)

    try:
        for web_page in range(2):
            interval = 1 
            prev_height = browser.execute_script("return document.body.scrollHeight")

            while True:
                browser.execute_script("window.scrollTo(0, document.body.scrollHeight)")
                time.sleep(interval)
                curr_height = browser.execute_script("return document.body.scrollHeight")
                if curr_height == prev_height:
                    break
                prev_height = curr_height

            #try:
                #browser.execute_script("arguments[0].click();", WebDriverWait(browser, 20).until(EC.element_to_be_clickable((By.CLASS_NAME,"collapse-button"))))
            #except Exception as e:
                #print(f"Error: {str(e)}")

        news_url = []
        
        hrefs = [element.get_attribute("href") for element in browser.find_elements(By.XPATH, "//*[@href]")]

        for href in hrefs:
            if href.endswith('.html'):
                news_url.append(href)

        news = pd.DataFrame(columns=['url', 'h1', 'text'])

        for url in news_url[:1]:
            if url.endswith('.html'):
                news_url.append(url)

            h1 = browser.find_element(By.TAG_NAME, 'h1')
            h1 = h1.text 

            p_tags = browser.find_elements(By.TAG_NAME, 'p')

            text_content = ""
            for p in p_tags:
                text_content += p.text + " "
            text_content = text_content.strip()

            news = news._append({'url': url, 'h1': ''.join(h1), 'text': text_content}, ignore_index=True)
            return news
        
    finally:
        browser.delete_all_cookies()
        browser.quit()

In [32]:
# Undetected Chromedriver took 35.5 seconds to scrape the news
# Incognito Chrome took 44.2 seconds to complete the same task

yf_news = get_yf_news("https://finance.yahoo.com/news/analysts-made-financial-statement-dws-060237470.html")

In [34]:
yf_news['text'][0]

"Last week, you might have seen that DWS Group GmbH & Co. KGaA (ETR:DWS) released its annual result to the market. The early response was not positive, with shares down 5.6% to €35.72 in the past week. Revenues of €2.6b were in line with forecasts, although statutory earnings per share (EPS) came in below expectations at €2.83, missing estimates by 3.0%. Following the result, the analysts have updated their earnings model, and it would be good to know whether they think there's been a strong change in the company's prospects, or if it's business as usual. We thought readers would find it interesting to see the analysts latest (statutory) post-earnings forecasts for next year. Check out our latest analysis for DWS Group GmbH KGaA Following last week's earnings report, DWS Group GmbH KGaA's eleven analysts are forecasting 2024 revenues to be €2.65b, approximately in line with the last 12 months. Statutory earnings per share are predicted to grow 11% to €3.13. Yet prior to the latest earn

* Wall Street Journal (Undetected Chromedriver is used)

In [35]:
def get_wsj_news(url):

    options = webdriver.ChromeOptions()
    # https://hidemy.io/en/proxy-list/ 에서 proxy server 가져오시면 되는데 스피드가 가장 좋은 서버를 선택하면 크롤링도 빨라지더라구요..!
    options.add_argument('proxy-server=172.67.134.0:80')
    options.add_argument("--disable-cache")
    browser = uc.Chrome(
        options = options
    )
    browser.get(url)
    browser.implicitly_wait(2)

    try:
        for web_page in range(2):
            interval = 1
            prev_height = browser.execute_script("return document.body.scrollHeight")

            while True:
                browser.execute_script("window.scrollTo(0, document.body.scrollHeight)")
                time.sleep(interval)
                curr_height = browser.execute_script("return document.body.scrollHeight")
                if curr_height == prev_height:
                    break
                prev_height = curr_height


        news_url = []
        
        hrefs = [element.get_attribute("href") for element in browser.find_elements(By.XPATH, "//*[@href]")]

        for href in hrefs:
            news_url.append(href)

        news = pd.DataFrame(columns=['url', 'h1', 'text'])

        for url in news_url[:1]:
            news_url.append(url)

            h1 = browser.find_element(By.TAG_NAME, 'h1')
            h1 = h1.text 

            p_tags = browser.find_elements(By.CLASS_NAME, 'e1e4oisd0')

            text_content = ""
            for p in p_tags:
                text_content += p.text + " "
            text_content = text_content.strip()

            news = news._append({'url': url, 'h1': ''.join(h1), 'text': text_content}, ignore_index=True)
            return news
        
    finally:
        browser.delete_all_cookies()
        browser.quit()

In [36]:
wsj_news = get_wsj_news('https://www.wsj.com/finance/investing/your-mutual-fund-stinks-can-this-wall-street-invention-change-that-73e86075?mod=finance_lead_pos5')

In [37]:
wsj_news

,url,h1,text
0,https://www.wsj.com/finance/investing/your-mut...,Your Mutual Fund Stinks. Can This Wall Street ...,"With ETFs, you can go along for the ride—or, p..."


In [39]:
wsj_news['text'][0]

'With ETFs, you can go along for the ride—or, perhaps, get taken for a ride. The typical exchange-traded fund is cheaper, performs better and produces lower tax bills than its ancestor, the mutual fund. As a result, investors have replaced mutual funds with ETFs en masse. Fund companies are feverishly working on plans to slow that outflow. At passive, market-tracking index funds, the ultimate result will probably be even lower costs and tax bills. At underperforming active funds, it could entrap disgruntled investors and keep them from voting with their feet. You’ll have to guard against the possibility that ETFs might not work for you—but against you.'

* Financial Times and New York Times (Embedded paywall + signup) [X]

In [62]:
import requests
from zenrows import ZenRowsClient
from bs4 import BeautifulSoup

client = ZenRowsClient("--API KEY--")
url = "https://www.ft.com/content/7a97e9c9-444f-4a5a-88ee-e5e9a8aea4d2"
params = {"js_render":"true","block_resources":"media,xhr,eventsource,image"}

response = client.get(url, params=params)
soup = BeautifulSoup(response.text, "html.parser")

ft_news = pd.DataFrame(columns=['URL','h1','text'])

title = soup.find("h1").text
p_tags = soup.find_all('p')

text_content = ""
for p in p_tags:
    text_content += p.text + " "
text_content = text_content.strip()

ft_news = ft_news._append(
    {"URL": url,
     "h1": title,
     "text": text_content},
    ignore_index=True
)


In [65]:
ft_news

,URL,h1,text
0,https://www.ft.com/content/7a97e9c9-444f-4a5a-...,HSBC caves to investor pressure on capital mar...,Try unlimited accessOnly C$1 for 4 weeks Then ...


* CNBC (Apify API is used to scrape publicly accessible content)

In [97]:
from apify_client import ApifyClient

# Initialize the ApifyClient with your API token
client = ApifyClient("--API KEY--")

# Prepare the Actor input with the extendOutputFunction field containing JavaScript code for web scraping
run_input = {
    "articleUrls": [{"url": "https://www.cnbc.com/2024/02/11/top-wall-street-analysts-pick-these-3-dividend-stocks-for-the-long-haul.html"}],
    "pageWaitMs": 100,
    "scrollToBottomMaxSecs": 2,
    "proxyConfiguration": {"useApifyProxy": True},
    "useGoogleBotHeaders": True
}

# Run the Actor and wait for it to finish
run = client.actor("lukaskrivka/article-extractor-smart").call(run_input=run_input)

# Fetch and print Actor results from the run's dataset (if there are any)
for item in client.dataset(run["defaultDatasetId"]).iterate_items():
    print(item)

{'url': 'https://www.cnbc.com/2024/02/11/top-wall-street-analysts-pick-these-3-dividend-stocks-for-the-long-haul.html', 'loadedUrl': 'https://www.cnbc.com/2024/02/11/top-wall-street-analysts-pick-these-3-dividend-stocks-for-the-long-haul.html', 'startUrl': 'https://www.cnbc.com/2024/02/11/top-wall-street-analysts-pick-these-3-dividend-stocks-for-the-long-haul.html', 'title': 'Top Wall Street analysts pick these 3 dividend stocks for the long haul', 'softTitle': 'Top Wall Street analysts pick these 3 dividend stocks for the long haul', 'date': '2024-02-11T11:38:35.000Z', 'author': ['TipRanks.com Staff', 'TipRanks.com Staff'], 'publisher': 'CNBC', 'copyright': '2024 CNBC LLC', 'description': "TipRanks' analyst ranking service pinpoints Wall Street's best-performing stocks, including Verizon and Enterprise Products Partners.", 'keywords': 'Enterprise Products Partners LP,Verizon Communications Inc,Investment strategy,Markets,Weekend brief,MPLX LP,Verizon Communications Inc,business news',

In [98]:
cnbc_news = pd.DataFrame(columns=['URL','h1','text'])
for item in client.dataset(run["defaultDatasetId"]).iterate_items():
    cnbc_news = cnbc_news._append(
        {"URL": item['url'],
         "h1": item['title'],
         "text": item['text']},
        ignore_index=True
    )
cnbc_news

,URL,h1,text
0,https://www.cnbc.com/2024/02/11/top-wall-stree...,Top Wall Street analysts pick these 3 dividend...,A strong fourth-quarter earnings season is und...


In [153]:
# Dropping any '\n' and '\' characters from the text
cnbc_news['text'][0] = cnbc_news['text'][0].replace('\n', ' ')
cnbc_news['text'][0] = cnbc_news['text'][0].encode().decode('unicode_escape')
# cnbc_news['text'][0] = cnbc_news['text'][0].replace("\\'","'")
# cnbc_news['text'][0] = cnbc_news['text'][0].split('\\')
# cnbc_news['text'][0] = ' '.join(cnbc_news['text'][0])
cnbc_news['text'] = cnbc_news['text'].iloc[0]
cnbc_news

C:\Users\USER\AppData\Local\Temp\ipykernel_13792\559380656.py:2: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  cnbc_news['text'][0] = cnbc_news['text'][0].replace('\n', ' ')
C:\Users\USER\AppData\Local\Temp\ipykernel_13792\559380656.py:3: Fu

,URL,h1,text
0,https://www.cnbc.com/2024/02/11/top-wall-stree...,Top Wall Street analysts pick these 3 dividend...,A strong fourth-quarter earnings season is und...


In [154]:
cnbc_news['text'].iloc[0]

'A strong fourth-quarter earnings season is underway, and it\'s time for dividend-paying companies to shine.Resilient dividend-paying companies can offer long-term growth potential and steady income. Investors ought to consider the insight of top Wall Street pros as they hunt for dividend stocks with solid fundamentals.Here are three attractive dividend stocks, according to Wall Street\'s top experts on TipRanks, a platform that ranks analysts based on their past performance.First up is telecom giant Verizon Communications (VZ), which recently reported its fourth-quarter results and impressed investors with the robust jump in wireless postpaid phone subscriber additions.In 2023, the company raised its dividend for the 17th consecutive year. Verizon\'s quarterly dividend of $0.665 per share (annualized dividend of $2.66), reflects a yield of 6.7%.Following Verizon\'s Q4 results, Tigress Financial analyst Ivan Feinseth reiterated a buy rating on the stock and increased the price target t

In [124]:
test = " the company\'s "
new_test = test.replace('\\\'', "'")
print(new_test)


 the company's 


In [130]:
test = " the company\'s "
new_test = test.split('\\')
new_test = ''.join(new_test).strip()
new_test

"the company's"

* Forbes (Apify API)

In [158]:
from apify_client import ApifyClient

# Initialize the ApifyClient with your API token
client = ApifyClient("--API KEY--")

# Prepare the Actor input with the extendOutputFunction field containing JavaScript code for web scraping
run_input = {
    "articleUrls": [{"url": "https://www.forbes.com/sites/karlfreund/2024/02/09/why-nvidia-is-entering-the-30b-market-for-custom-chips/?sh=39ac09bd6d290"}],
    "pageWaitMs": 100,
    "scrollToBottomMaxSecs": 2,
    "proxyConfiguration": {"useApifyProxy": True},
    "useGoogleBotHeaders": True
}

# Run the Actor and wait for it to finish
run = client.actor("lukaskrivka/article-extractor-smart").call(run_input=run_input)

fb_news = pd.DataFrame(columns=['URL','h1','text'])
for item in client.dataset(run["defaultDatasetId"]).iterate_items():
    fb_news = fb_news._append(
        {"URL": item['url'],
         "h1": item['title'],
         "text": item['text']},
        ignore_index=True
    )
    
fb_news

,URL,h1,text
0,https://www.forbes.com/sites/karlfreund/2024/0...,Why Nvidia Is Entering The $30B Market For Cus...,"Nvidia’s largest customers (e.g., Google, Amaz..."


In [162]:
fb_news['text'][0] = fb_news['text'][0].replace('\n', ' ')
fb_news['text'] = fb_news['text'].iloc[0]
fb_news['text'][0]

'Nvidia’s largest customers (e.g., Google, Amazon, Microsoft, Meta, and OpenAI) are developing AI chips that compete with Nvidia, presenting a potential long-term threat to the AI leader. Jensen’s response? “We can help you do that.”  Warning: this blog contains speculation that has not been verified.... But fun to think about!  When I was working at AMD, I was always impressed with how the company managed to keep two teams in complete isolation from each other to protect client confidentiality. One team was designing the next chip for the Microsoft XBox, while the other was designing a chip for the Sony Playstation. Each client had their own gaming console intellectual property and requirements which had to be protected from the other team. It was a successful model for AMD, who still owns that market.  But all that secrecy can be difficult and expensive. And it is hard to scale that business. What if the chip vendor let the customer do more of the design work, and provide their IP fo

* The Economist

* Reuters